In [242]:
import pandas as pd
import seaborn as sns
import numpy as np
import os.path
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.preprocessing import power_transform, MinMaxScaler, StandardScaler
from imblearn import under_sampling, over_sampling
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier


# import os
# os.getcwd()
linux_dir = '/home/satan/Projects/Dataset/HCI/home-credit-default-risk/'
win_dir = 'D:/Projects/Dataset/HCI/home-credit-default-risk/'

In [114]:
app_col = ['SK_ID_CURR','NAME_CONTRACT_TYPE','CODE_GENDER','FLAG_OWN_REALTY','CNT_CHILDREN','AMT_INCOME_TOTAL','AMT_CREDIT',
           'AMT_ANNUITY','AMT_GOODS_PRICE','NAME_INCOME_TYPE','NAME_EDUCATION_TYPE','NAME_HOUSING_TYPE','OWN_CAR_AGE','OCCUPATION_TYPE',
           'CNT_FAM_MEMBERS','REGION_RATING_CLIENT_W_CITY','AMT_REQ_CREDIT_BUREAU_QRT','NAME_FAMILY_STATUS']

prev_app_col = ['SK_ID_CURR','SK_ID_PREV','NAME_CONTRACT_TYPE','AMT_ANNUITY','AMT_APPLICATION','AMT_CREDIT',
                'NAME_CONTRACT_STATUS','DAYS_DECISION','NAME_CLIENT_TYPE','CNT_PAYMENT','DAYS_FIRST_DUE',
                'DAYS_LAST_DUE_1ST_VERSION','DAYS_LAST_DUE','DAYS_TERMINATION','NFLAG_INSURED_ON_APPROVAL','NAME_YIELD_GROUP']
prev_app_cat = ['NAME_CASH_LOAN_PURPOSE','CODE_REJECT_REASON']
installment_col = ['SK_ID_PREV','SK_ID_CURR','DAYS_INSTALMENT','DAYS_ENTRY_PAYMENT','AMT_INSTALMENT','AMT_PAYMENT']


In [232]:
def kdeplot_me(df, columns, hue=None, figsize=(30, 15)):
    plt.figure(figsize=figsize)
    for col_name, index in zip(columns, range(len(columns))):
        plt.subplot( int(len(columns)/6)+1  , 6, index+1)
        if hue != None:
            sns.kdeplot(data=df, x=col_name, hue=hue, color='green', linewidth=3, fill=True)
        else:
            sns.kdeplot(data=df, x=col_name, color='green', linewidth=3, fill=True)
        plt.xlabel(col_name)
    plt.show()

def violinplot_me(df, columns, hue=None, figsize=(30, 15)):
    plt.figure(figsize=figsize)
    for col_name, index in zip(columns, range(len(columns))):
        plt.subplot( int(len(columns)/6)+1  , 6, index+1)
        if hue != None:
            sns.violinplot(data=df, x=col_name, hue=hue, color='green', linewidth=3, fill=True)
        else:
            sns.violinplot(data=df, x=col_name, color='green', linewidth=3, fill=True)
        plt.xlabel(col_name)
    plt.show()
    
def boxplot_me(df, columns, x, figsize=(30, 15)):
    plt.figure(figsize=figsize)
    for col_name, index in zip(columns, range(len(columns))):
        plt.subplot( int(len(columns)/6)+1  , 6, index+1)
        sns.boxplot(data=df, x=x, y=col_name)
        plt.xlabel(col_name)
    plt.show()
       
def log_transform(temp_df, log_columns):
    for col in log_columns:
        if temp_df[col].min() == 0:
            # df[col] = np.log(df[col]+1)
            temp_df[col], fitted_lambda = stats.boxcox( ( temp_df[col] + 1) )
        elif df[col].min() > 0:
            # df[col] = np.log(df[col])
            temp_df[col], fitted_lambda = stats.boxcox( temp_df[col] )

def get_minmax_scaler(temp_df, columns):
    scaler = {}
    for col in columns:
        scaler[col] = MinMaxScaler().fit( np.array(temp_df[col]).reshape(-1,1) )
    return scaler

def get_standard_scaler(temp_df, columns):
    scaler = {}
    for col in columns:
        scaler[col] = StandardScaler().fit( np.array(temp_df[col]).reshape(-1,1) )
    return scaler

def apply_transform(scaler, temp_df):
    tmp_df = temp_df.copy()
    for col in scaler.keys():
        tmp_df[col] = scaler[col].transform( np.array(temp_df[col]).reshape(-1,1) )
    return tmp_df


### **Application Train|Test ✓**

In [116]:
try :
    app_train = pd.read_csv(linux_dir+'application_train.csv')
    app_test = pd.read_csv(linux_dir+'application_test.csv')
except:
    app_train = pd.read_csv(win_dir+'application_train.csv')
    app_test = pd.read_csv(win_dir+'application_test.csv')

In [117]:
app_train.OWN_CAR_AGE.fillna(0, inplace=True)
app_test.OWN_CAR_AGE.fillna(0, inplace=True)
app_train.loc[ app_train.FLAG_OWN_CAR == 'N', 'OWN_CAR_AGE' ] = -1
app_test.loc[ app_train.FLAG_OWN_CAR == 'N', 'OWN_CAR_AGE' ] = -1

app_train_tmp, app_test_tmp = app_train[app_col+['TARGET']].copy(), app_test[app_col].copy()
app_train_tmp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 19 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   SK_ID_CURR                   307511 non-null  int64  
 1   NAME_CONTRACT_TYPE           307511 non-null  object 
 2   CODE_GENDER                  307511 non-null  object 
 3   FLAG_OWN_REALTY              307511 non-null  object 
 4   CNT_CHILDREN                 307511 non-null  int64  
 5   AMT_INCOME_TOTAL             307511 non-null  float64
 6   AMT_CREDIT                   307511 non-null  float64
 7   AMT_ANNUITY                  307499 non-null  float64
 8   AMT_GOODS_PRICE              307233 non-null  float64
 9   NAME_INCOME_TYPE             307511 non-null  object 
 10  NAME_EDUCATION_TYPE          307511 non-null  object 
 11  NAME_HOUSING_TYPE            307511 non-null  object 
 12  OWN_CAR_AGE                  307511 non-null  float64
 13 

In [118]:
# for c in app_train.columns[app_train.dtypes == 'object']:
#     c_train = set(app_train[c].unique())
#     c_test = set(app_test[c].unique())
#     diff = c_train ^ c_test
#     if len(diff) > 0:
#         print('feature ' + c + ' has different values: ', diff)

app_train_tmp.drop(index=app_train_tmp[(app_train_tmp.NAME_FAMILY_STATUS=='Unknown')|(app_train_tmp.CODE_GENDER=='XNA')|(app_train_tmp.NAME_INCOME_TYPE=='Maternity leave')].index, inplace=True)

In [119]:
app_train_tmp['TARGET'].value_counts(normalize=True)*100

0    91.92748
1     8.07252
Name: TARGET, dtype: float64

In [120]:
app_cat_var = app_train_tmp.columns[app_train_tmp.dtypes == 'object'].tolist()
for col in app_cat_var:
    print(col,'\n',app_train_tmp[col].value_counts(),'\n')

NAME_CONTRACT_TYPE 
 Cash loans         278230
Revolving loans     29270
Name: NAME_CONTRACT_TYPE, dtype: int64 

CODE_GENDER 
 F    202443
M    105057
Name: CODE_GENDER, dtype: int64 

FLAG_OWN_REALTY 
 Y    213302
N     94198
Name: FLAG_OWN_REALTY, dtype: int64 

NAME_INCOME_TYPE 
 Working                 158771
Commercial associate     71614
Pensioner                55362
State servant            21703
Unemployed                  22
Student                     18
Businessman                 10
Name: NAME_INCOME_TYPE, dtype: int64 

NAME_EDUCATION_TYPE 
 Secondary / secondary special    218387
Higher education                  74858
Incomplete higher                 10276
Lower secondary                    3815
Academic degree                     164
Name: NAME_EDUCATION_TYPE, dtype: int64 

NAME_HOUSING_TYPE 
 House / apartment      272859
With parents            14840
Municipal apartment     11181
Rented apartment         4881
Office apartment         2617
Co-op apartment          

In [121]:
app_cat_binary = ['NAME_CONTRACT_TYPE','CODE_GENDER','FLAG_OWN_REALTY']
app_cat_multi = [col for col in app_cat_var if col not in app_cat_binary + ['NAME_EDUCATION_TYPE']]
edu_map = {'Lower secondary':0, 'Secondary / secondary special': 1, 'Incomplete higher':2, 'Higher education':3, 'Academic degree':4}

for col in app_cat_binary:
    temp_dict = dict([ [i,j] for i,j in zip(app_train_tmp[col].value_counts().index.to_list(),[0,1])])
    app_train_tmp[col] = app_train_tmp[col].map(temp_dict)
    app_test_tmp[col] = app_train_tmp[col].map(temp_dict)
    
app_train_tmp['NAME_EDUCATION_TYPE'] = app_train_tmp['NAME_EDUCATION_TYPE'].map(edu_map) 
app_test_tmp['NAME_EDUCATION_TYPE'] = app_test_tmp['NAME_EDUCATION_TYPE'].map(edu_map) 

In [122]:
app_train_tmp = pd.get_dummies(app_train_tmp, columns=app_cat_multi, drop_first=True)
app_test_tmp = pd.get_dummies(app_test_tmp, columns=app_cat_multi, drop_first=True)

In [123]:
app_train_tmp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 307500 entries, 0 to 307510
Data columns (total 47 columns):
 #   Column                                   Non-Null Count   Dtype  
---  ------                                   --------------   -----  
 0   SK_ID_CURR                               307500 non-null  int64  
 1   NAME_CONTRACT_TYPE                       307500 non-null  int64  
 2   CODE_GENDER                              307500 non-null  int64  
 3   FLAG_OWN_REALTY                          307500 non-null  int64  
 4   CNT_CHILDREN                             307500 non-null  int64  
 5   AMT_INCOME_TOTAL                         307500 non-null  float64
 6   AMT_CREDIT                               307500 non-null  float64
 7   AMT_ANNUITY                              307488 non-null  float64
 8   AMT_GOODS_PRICE                          307224 non-null  float64
 9   NAME_EDUCATION_TYPE                      307500 non-null  int64  
 10  OWN_CAR_AGE                     

### **Bureau**

In [124]:
try:
    bureau = pd.read_csv(linux_dir+'bureau.csv')
except:
    bureau = pd.read_csv(win_dir+'bureau.csv')
bureau.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1716428 entries, 0 to 1716427
Data columns (total 17 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   SK_ID_CURR              1716428 non-null  int64  
 1   SK_ID_BUREAU            1716428 non-null  int64  
 2   CREDIT_ACTIVE           1716428 non-null  object 
 3   CREDIT_CURRENCY         1716428 non-null  object 
 4   DAYS_CREDIT             1716428 non-null  int64  
 5   CREDIT_DAY_OVERDUE      1716428 non-null  int64  
 6   DAYS_CREDIT_ENDDATE     1610875 non-null  float64
 7   DAYS_ENDDATE_FACT       1082775 non-null  float64
 8   AMT_CREDIT_MAX_OVERDUE  591940 non-null   float64
 9   CNT_CREDIT_PROLONG      1716428 non-null  int64  
 10  AMT_CREDIT_SUM          1716415 non-null  float64
 11  AMT_CREDIT_SUM_DEBT     1458759 non-null  float64
 12  AMT_CREDIT_SUM_LIMIT    1124648 non-null  float64
 13  AMT_CREDIT_SUM_OVERDUE  1716428 non-null  float64
 14  CR

In [125]:
for col in bureau.columns[ bureau.dtypes=='object']:
    print(col,'\n',bureau[col].value_counts(),'\n')
credit_type_others = [type_index for type_index, type_count in zip(bureau['CREDIT_TYPE'].value_counts().index, bureau['CREDIT_TYPE'].value_counts().values) if type_count < 1500]

CREDIT_ACTIVE 
 Closed      1079273
Active       630607
Sold           6527
Bad debt         21
Name: CREDIT_ACTIVE, dtype: int64 

CREDIT_CURRENCY 
 currency 1    1715020
currency 2       1224
currency 3        174
currency 4         10
Name: CREDIT_CURRENCY, dtype: int64 

CREDIT_TYPE 
 Consumer credit                                 1251615
Credit card                                      402195
Car loan                                          27690
Mortgage                                          18391
Microloan                                         12413
Loan for business development                      1975
Another type of loan                               1017
Unknown type of loan                                555
Loan for working capital replenishment              469
Cash loan (non-earmarked)                            56
Real estate loan                                     27
Loan for the purchase of equipment                   19
Loan for purchase of shares (margin le

In [126]:
# bureau.columns[ (bureau.dtypes != 'object') & (~bureau.columns.isin(['SK_ID_CURR', 'SK_ID_BUREAU'])) ].to_list()
# sns.kdeplot(bureau, bureau.columns[ (bureau.dtypes != 'object') & (~bureau.columns.isin(['SK_ID_CURR', 'SK_ID_BUREAU'])) ].to_list() )
# bureau[bureau.columns[ (bureau.dtypes != 'object') & (~bureau.columns.isin(['SK_ID_CURR', 'SK_ID_BUREAU'])) ].to_list()]

In [127]:
bureau.loc[ bureau.CREDIT_TYPE.isin(credit_type_others), 'CREDIT_TYPE' ] = 'Others'
bureau = pd.get_dummies(bureau, columns=bureau.columns[ bureau.dtypes=='object'], drop_first=True)
bureau.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1716428 entries, 0 to 1716427
Data columns (total 26 columns):
 #   Column                                     Non-Null Count    Dtype  
---  ------                                     --------------    -----  
 0   SK_ID_CURR                                 1716428 non-null  int64  
 1   SK_ID_BUREAU                               1716428 non-null  int64  
 2   DAYS_CREDIT                                1716428 non-null  int64  
 3   CREDIT_DAY_OVERDUE                         1716428 non-null  int64  
 4   DAYS_CREDIT_ENDDATE                        1610875 non-null  float64
 5   DAYS_ENDDATE_FACT                          1082775 non-null  float64
 6   AMT_CREDIT_MAX_OVERDUE                     591940 non-null   float64
 7   CNT_CREDIT_PROLONG                         1716428 non-null  int64  
 8   AMT_CREDIT_SUM                             1716415 non-null  float64
 9   AMT_CREDIT_SUM_DEBT                        1458759 non-null  float64

### **Bureau Balance**

In [128]:
try :
    bureau_bal = pd.read_csv(linux_dir+'bureau_balance.csv')
except:
    bureau_bal = pd.read_csv(win_dir+'bureau_balance.csv')

bureau_bal.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27299925 entries, 0 to 27299924
Data columns (total 3 columns):
 #   Column          Non-Null Count     Dtype 
---  ------          --------------     ----- 
 0   SK_ID_BUREAU    27299925 non-null  int64 
 1   MONTHS_BALANCE  27299925 non-null  int64 
 2   STATUS          27299925 non-null  object
dtypes: int64(2), object(1)
memory usage: 624.8+ MB


In [129]:
bureau_bal = pd.get_dummies(bureau_bal, columns=['STATUS'], drop_first=True)
bureau_bal

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS_1,STATUS_2,STATUS_3,STATUS_4,STATUS_5,STATUS_C,STATUS_X
0,5715448,0,0,0,0,0,0,1,0
1,5715448,-1,0,0,0,0,0,1,0
2,5715448,-2,0,0,0,0,0,1,0
3,5715448,-3,0,0,0,0,0,1,0
4,5715448,-4,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...
27299920,5041336,-47,0,0,0,0,0,0,1
27299921,5041336,-48,0,0,0,0,0,0,1
27299922,5041336,-49,0,0,0,0,0,0,1
27299923,5041336,-50,0,0,0,0,0,0,1


In [130]:
for col in [col for col in bureau_bal.columns if col not in ['SK_ID_BUREAU']]:
    print(col)
    agg_mean = bureau_bal.groupby(['SK_ID_BUREAU'])[col].mean()
    bureau[col+'_mean'] = bureau_bal['SK_ID_BUREAU'].map(lambda x: agg_mean[x] if x in agg_mean.index else np.nan)
bureau.info(verbose=True, show_counts=True)

MONTHS_BALANCE
STATUS_1
STATUS_2
STATUS_3
STATUS_4
STATUS_5
STATUS_C
STATUS_X
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1716428 entries, 0 to 1716427
Data columns (total 34 columns):
 #   Column                                     Non-Null Count    Dtype  
---  ------                                     --------------    -----  
 0   SK_ID_CURR                                 1716428 non-null  int64  
 1   SK_ID_BUREAU                               1716428 non-null  int64  
 2   DAYS_CREDIT                                1716428 non-null  int64  
 3   CREDIT_DAY_OVERDUE                         1716428 non-null  int64  
 4   DAYS_CREDIT_ENDDATE                        1610875 non-null  float64
 5   DAYS_ENDDATE_FACT                          1082775 non-null  float64
 6   AMT_CREDIT_MAX_OVERDUE                     591940 non-null   float64
 7   CNT_CREDIT_PROLONG                         1716428 non-null  int64  
 8   AMT_CREDIT_SUM                             1716415 non-null  flo

Aggregating Bureau => Application_Train

In [131]:
for col in [col for col in bureau.columns if col not in ['SK_ID_CURR','SK_ID_BUREAU']]:
    print(col)
    agg_mean = bureau.groupby(['SK_ID_CURR'])[col].mean()
    app_train_tmp['BUR_'+col+'_mean'] = bureau['SK_ID_CURR'].map(lambda x: agg_mean[x] if x in agg_mean.index else np.nan)
app_train_tmp.info(verbose=True, show_counts=True)

DAYS_CREDIT
CREDIT_DAY_OVERDUE
DAYS_CREDIT_ENDDATE
DAYS_ENDDATE_FACT
AMT_CREDIT_MAX_OVERDUE
CNT_CREDIT_PROLONG
AMT_CREDIT_SUM
AMT_CREDIT_SUM_DEBT
AMT_CREDIT_SUM_LIMIT
AMT_CREDIT_SUM_OVERDUE
DAYS_CREDIT_UPDATE
AMT_ANNUITY
CREDIT_ACTIVE_Bad debt
CREDIT_ACTIVE_Closed
CREDIT_ACTIVE_Sold
CREDIT_CURRENCY_currency 2
CREDIT_CURRENCY_currency 3
CREDIT_CURRENCY_currency 4
CREDIT_TYPE_Consumer credit
CREDIT_TYPE_Credit card
CREDIT_TYPE_Loan for business development
CREDIT_TYPE_Microloan
CREDIT_TYPE_Mortgage
CREDIT_TYPE_Others
MONTHS_BALANCE_mean
STATUS_1_mean
STATUS_2_mean
STATUS_3_mean
STATUS_4_mean
STATUS_5_mean
STATUS_C_mean
STATUS_X_mean
<class 'pandas.core.frame.DataFrame'>
Int64Index: 307500 entries, 0 to 307510
Data columns (total 79 columns):
 #   Column                                              Non-Null Count   Dtype  
---  ------                                              --------------   -----  
 0   SK_ID_CURR                                          307500 non-null  int64  
 1  

### **Previous Application ✓**

In [132]:
try:
    prev_app = pd.read_csv(linux_dir+'previous_application.csv')
except:
    prev_app = pd.read_csv(win_dir+'previous_application.csv')

prev_app.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1670214 entries, 0 to 1670213
Data columns (total 37 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   SK_ID_PREV                   1670214 non-null  int64  
 1   SK_ID_CURR                   1670214 non-null  int64  
 2   NAME_CONTRACT_TYPE           1670214 non-null  object 
 3   AMT_ANNUITY                  1297979 non-null  float64
 4   AMT_APPLICATION              1670214 non-null  float64
 5   AMT_CREDIT                   1670213 non-null  float64
 6   AMT_DOWN_PAYMENT             774370 non-null   float64
 7   AMT_GOODS_PRICE              1284699 non-null  float64
 8   WEEKDAY_APPR_PROCESS_START   1670214 non-null  object 
 9   HOUR_APPR_PROCESS_START      1670214 non-null  int64  
 10  FLAG_LAST_APPL_PER_CONTRACT  1670214 non-null  object 
 11  NFLAG_LAST_APPL_IN_DAY       1670214 non-null  int64  
 12  RATE_DOWN_PAYMENT            774370 non-nu

In [133]:
prev_app_tmp = prev_app[prev_app_col].copy()
prev_app_col

['SK_ID_CURR',
 'SK_ID_PREV',
 'NAME_CONTRACT_TYPE',
 'AMT_ANNUITY',
 'AMT_APPLICATION',
 'AMT_CREDIT',
 'NAME_CONTRACT_STATUS',
 'DAYS_DECISION',
 'NAME_CLIENT_TYPE',
 'CNT_PAYMENT',
 'DAYS_FIRST_DUE',
 'DAYS_LAST_DUE_1ST_VERSION',
 'DAYS_LAST_DUE',
 'DAYS_TERMINATION',
 'NFLAG_INSURED_ON_APPROVAL',
 'NAME_YIELD_GROUP']

In [134]:
for i in prev_app_tmp.columns[ prev_app_tmp.dtypes=='object']:
    print(i,'\n',prev_app_tmp[i].value_counts(),'\n')

NAME_CONTRACT_TYPE 
 Cash loans         747553
Consumer loans     729151
Revolving loans    193164
XNA                   346
Name: NAME_CONTRACT_TYPE, dtype: int64 

NAME_CONTRACT_STATUS 
 Approved        1036781
Canceled         316319
Refused          290678
Unused offer      26436
Name: NAME_CONTRACT_STATUS, dtype: int64 

NAME_CLIENT_TYPE 
 Repeater     1231261
New           301363
Refreshed     135649
XNA             1941
Name: NAME_CLIENT_TYPE, dtype: int64 

NAME_YIELD_GROUP 
 XNA           517215
middle        385532
high          353331
low_normal    322095
low_action     92041
Name: NAME_YIELD_GROUP, dtype: int64 



In [135]:
prev_app_tmp = pd.get_dummies(prev_app_tmp, columns=prev_app_tmp.columns[ prev_app_tmp.dtypes=='object'], drop_first=True)
prev_app_tmp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1670214 entries, 0 to 1670213
Data columns (total 25 columns):
 #   Column                              Non-Null Count    Dtype  
---  ------                              --------------    -----  
 0   SK_ID_CURR                          1670214 non-null  int64  
 1   SK_ID_PREV                          1670214 non-null  int64  
 2   AMT_ANNUITY                         1297979 non-null  float64
 3   AMT_APPLICATION                     1670214 non-null  float64
 4   AMT_CREDIT                          1670213 non-null  float64
 5   DAYS_DECISION                       1670214 non-null  int64  
 6   CNT_PAYMENT                         1297984 non-null  float64
 7   DAYS_FIRST_DUE                      997149 non-null   float64
 8   DAYS_LAST_DUE_1ST_VERSION           997149 non-null   float64
 9   DAYS_LAST_DUE                       997149 non-null   float64
 10  DAYS_TERMINATION                    997149 non-null   float64
 11  NFLAG_INSUR

In [136]:
# prev_app_temp['SK_ID_CURR'].value_counts()

In [137]:
# filtered_prev[ filtered_prev.SK_ID_CURR == 187868 ]['NAME_CONTRACT_TYPE'].value_counts()
# NFLAG_LAST_APPL_IN_DAY
prev_app_tmp[ (prev_app_tmp.SK_ID_CURR == 187868) ].iloc[:,10:]

# filtered_prev.FLAG_LAST_APPL_PER_CONTRACT.value_counts()

,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,NAME_CONTRACT_TYPE_Consumer loans,NAME_CONTRACT_TYPE_Revolving loans,NAME_CONTRACT_TYPE_XNA,NAME_CONTRACT_STATUS_Canceled,NAME_CONTRACT_STATUS_Refused,NAME_CONTRACT_STATUS_Unused offer,NAME_CLIENT_TYPE_Refreshed,NAME_CLIENT_TYPE_Repeater,NAME_CLIENT_TYPE_XNA,NAME_YIELD_GROUP_high,NAME_YIELD_GROUP_low_action,NAME_YIELD_GROUP_low_normal,NAME_YIELD_GROUP_middle
37305,NaN,NaN,0,0,0,0,1,0,0,1,0,0,0,0,0
56117,NaN,NaN,0,0,0,0,1,0,0,1,0,0,0,0,0
67781,NaN,NaN,0,0,0,0,1,0,0,1,0,0,0,0,0
92787,NaN,NaN,0,0,0,0,1,0,0,1,0,0,0,0,0
170959,NaN,NaN,0,0,0,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1552993,NaN,NaN,0,0,0,0,1,0,0,1,0,0,0,0,0
1575005,NaN,NaN,0,0,0,0,1,0,0,1,0,0,0,0,0
1581066,NaN,NaN,0,0,0,0,1,0,0,1,0,0,0,0,0
1636755,NaN,NaN,0,0,0,0,1,0,0,1,0,0,0,0,0


TOO MANY NA HAHAHA

In [138]:
# prev_app[['NAME_PAYMENT_TYPE']].value_counts()
# prev_app[['NAME_CASH_LOAN_PURPOSE']].value_counts()
# prev_app[['NAME_PORTFOLIO']].value_counts()
# prev_app[['CODE_REJECT_REASON']].value_counts()
# prev_app[['NAME_GOODS_CATEGORY']].value_counts()
# prev_app[['NAME_PRODUCT_TYPE']].value_counts()

# prev_app.groupby(['SK_ID_CURR']).agg(uniq=('NAME_PAYMENT_TYPE','nunique'))
# prev_app[ prev_app.SK_ID_CURR == 100003 ][['SK_ID_CURR','NAME_PAYMENT_TYPE']]
prev_app['DAYS_FIRST_DRAWING'].value_counts()

 365243.0    934444
-228.0          123
-224.0          121
-212.0          121
-223.0          119
              ...  
-1854.0           1
-1990.0           1
-1995.0           1
-1687.0           1
-1879.0           1
Name: DAYS_FIRST_DRAWING, Length: 2838, dtype: int64

### **POS Cash Balance ✓✅**

In [139]:
try:
    poscash_bal = pd.read_csv(linux_dir+'POS_CASH_balance.csv')
except:
    poscash_bal = pd.read_csv(win_dir+'POS_CASH_balance.csv')

poscash_bal.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10001358 entries, 0 to 10001357
Data columns (total 8 columns):
 #   Column                 Non-Null Count     Dtype  
---  ------                 --------------     -----  
 0   SK_ID_PREV             10001358 non-null  int64  
 1   SK_ID_CURR             10001358 non-null  int64  
 2   MONTHS_BALANCE         10001358 non-null  int64  
 3   CNT_INSTALMENT         9975287 non-null   float64
 4   CNT_INSTALMENT_FUTURE  9975271 non-null   float64
 5   NAME_CONTRACT_STATUS   10001358 non-null  object 
 6   SK_DPD                 10001358 non-null  int64  
 7   SK_DPD_DEF             10001358 non-null  int64  
dtypes: float64(2), int64(5), object(1)
memory usage: 610.4+ MB


In [140]:
poscash_bal.drop(index=poscash_bal[(poscash_bal.NAME_CONTRACT_STATUS.isin(['XNA']))].index, inplace=True)
# poshcash_bal['NAME_CONTRACT_STATUS'].value_counts()
poscash_bal = pd.get_dummies(poscash_bal, columns=['NAME_CONTRACT_STATUS'], drop_first=True)
poscash_bal.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10001356 entries, 0 to 10001357
Data columns (total 14 columns):
 #   Column                                      Non-Null Count     Dtype  
---  ------                                      --------------     -----  
 0   SK_ID_PREV                                  10001356 non-null  int64  
 1   SK_ID_CURR                                  10001356 non-null  int64  
 2   MONTHS_BALANCE                              10001356 non-null  int64  
 3   CNT_INSTALMENT                              9975287 non-null   float64
 4   CNT_INSTALMENT_FUTURE                       9975271 non-null   float64
 5   SK_DPD                                      10001356 non-null  int64  
 6   SK_DPD_DEF                                  10001356 non-null  int64  
 7   NAME_CONTRACT_STATUS_Amortized debt         10001356 non-null  uint8  
 8   NAME_CONTRACT_STATUS_Approved               10001356 non-null  uint8  
 9   NAME_CONTRACT_STATUS_Canceled               

In [141]:
for col in [col for col in poscash_bal.columns if col not in ['SK_ID_PREV','SK_ID_CURR']]:
    print(col)
    agg_mean = poscash_bal.groupby(['SK_ID_PREV'])[col].mean()
    prev_app_tmp['POS_'+col+'_mean'] = prev_app_tmp['SK_ID_PREV'].map(lambda x: agg_mean[x] if x in agg_mean.index else np.nan)
prev_app_tmp.info()

MONTHS_BALANCE
CNT_INSTALMENT
CNT_INSTALMENT_FUTURE
SK_DPD
SK_DPD_DEF
NAME_CONTRACT_STATUS_Amortized debt
NAME_CONTRACT_STATUS_Approved
NAME_CONTRACT_STATUS_Canceled
NAME_CONTRACT_STATUS_Completed
NAME_CONTRACT_STATUS_Demand
NAME_CONTRACT_STATUS_Returned to the store
NAME_CONTRACT_STATUS_Signed
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1670214 entries, 0 to 1670213
Data columns (total 37 columns):
 #   Column                                               Non-Null Count    Dtype  
---  ------                                               --------------    -----  
 0   SK_ID_CURR                                           1670214 non-null  int64  
 1   SK_ID_PREV                                           1670214 non-null  int64  
 2   AMT_ANNUITY                                          1297979 non-null  float64
 3   AMT_APPLICATION                                      1670214 non-null  float64
 4   AMT_CREDIT                                           1670213 non-null  float64
 5 

### **Installment Payments ✓✅**

In [142]:
try:
    install_payment = pd.read_csv(linux_dir+'installments_payments.csv')
except:
    install_payment = pd.read_csv(win_dir+'installments_payments.csv')

install_payment.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13605401 entries, 0 to 13605400
Data columns (total 8 columns):
 #   Column                  Non-Null Count     Dtype  
---  ------                  --------------     -----  
 0   SK_ID_PREV              13605401 non-null  int64  
 1   SK_ID_CURR              13605401 non-null  int64  
 2   NUM_INSTALMENT_VERSION  13605401 non-null  float64
 3   NUM_INSTALMENT_NUMBER   13605401 non-null  int64  
 4   DAYS_INSTALMENT         13605401 non-null  float64
 5   DAYS_ENTRY_PAYMENT      13602496 non-null  float64
 6   AMT_INSTALMENT          13605401 non-null  float64
 7   AMT_PAYMENT             13602496 non-null  float64
dtypes: float64(5), int64(3)
memory usage: 830.4 MB


In [143]:
for col in [col for col in install_payment.columns if col not in ['SK_ID_PREV','SK_ID_CURR']]:
    agg_mean = install_payment.groupby(['SK_ID_PREV'])[col].mean()
    prev_app_tmp['IP_'+col+'_mean'] = prev_app_tmp['SK_ID_PREV'].map(lambda x: agg_mean[x] if x in agg_mean.index else np.nan)

In [144]:
install_pay_tmp = install_payment.groupby(['SK_ID_PREV'])

In [145]:
# for col in ['NUM_INSTALMENT_VERSION','NUM_INSTALMENT_NUMBER']:
#     print(col,'\n',install_payment[col].value_counts(),'\n')

In [146]:
# kdeplot_me(install_payment, [col for col in install_payment.columns if col not in ['SK_ID_PREV','SK_ID_CURR']])

In [147]:
# install_payment.groupby(['SK_ID_CURR','SK_ID_PREV']).size()

### **Credit Card Balance ✓ ✅**

In [148]:
try:
    cc_bal = pd.read_csv(linux_dir+'credit_card_balance.csv')
except:
    cc_bal = pd.read_csv(win_dir+'credit_card_balance.csv')
    
cc_bal.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3840312 entries, 0 to 3840311
Data columns (total 23 columns):
 #   Column                      Non-Null Count    Dtype  
---  ------                      --------------    -----  
 0   SK_ID_PREV                  3840312 non-null  int64  
 1   SK_ID_CURR                  3840312 non-null  int64  
 2   MONTHS_BALANCE              3840312 non-null  int64  
 3   AMT_BALANCE                 3840312 non-null  float64
 4   AMT_CREDIT_LIMIT_ACTUAL     3840312 non-null  int64  
 5   AMT_DRAWINGS_ATM_CURRENT    3090496 non-null  float64
 6   AMT_DRAWINGS_CURRENT        3840312 non-null  float64
 7   AMT_DRAWINGS_OTHER_CURRENT  3090496 non-null  float64
 8   AMT_DRAWINGS_POS_CURRENT    3090496 non-null  float64
 9   AMT_INST_MIN_REGULARITY     3535076 non-null  float64
 10  AMT_PAYMENT_CURRENT         3072324 non-null  float64
 11  AMT_PAYMENT_TOTAL_CURRENT   3840312 non-null  float64
 12  AMT_RECEIVABLE_PRINCIPAL    3840312 non-null  float64
 1

In [149]:
# for col in cc_bal.columns:
#     print(col,'\n',cc_bal[col].value_counts(),'\n')

cc_col = [col for col in cc_bal.columns if col not in ['SK_ID_PREV','SK_ID_CURR','NAME_CONTRACT_STATUS']]
# kdeplot_me(cc_bal, cc_col)
# cc_bal['NAME_CONTRACT_STATUS'].value_counts()
# pd.get_dummies(cc_bal, columns=['NAME_CONTRACT_STATUS'], drop_first=True).info()
cc_bal = pd.get_dummies(cc_bal, columns=['NAME_CONTRACT_STATUS'], drop_first=True)
cc_bal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3840312 entries, 0 to 3840311
Data columns (total 28 columns):
 #   Column                              Dtype  
---  ------                              -----  
 0   SK_ID_PREV                          int64  
 1   SK_ID_CURR                          int64  
 2   MONTHS_BALANCE                      int64  
 3   AMT_BALANCE                         float64
 4   AMT_CREDIT_LIMIT_ACTUAL             int64  
 5   AMT_DRAWINGS_ATM_CURRENT            float64
 6   AMT_DRAWINGS_CURRENT                float64
 7   AMT_DRAWINGS_OTHER_CURRENT          float64
 8   AMT_DRAWINGS_POS_CURRENT            float64
 9   AMT_INST_MIN_REGULARITY             float64
 10  AMT_PAYMENT_CURRENT                 float64
 11  AMT_PAYMENT_TOTAL_CURRENT           float64
 12  AMT_RECEIVABLE_PRINCIPAL            float64
 13  AMT_RECIVABLE                       float64
 14  AMT_TOTAL_RECEIVABLE                float64
 15  CNT_DRAWINGS_ATM_CURRENT            float64
 16  

In [192]:
cc_bal.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3840312 entries, 0 to 3840311
Data columns (total 28 columns):
 #   Column                              Non-Null Count    Dtype  
---  ------                              --------------    -----  
 0   SK_ID_PREV                          3840312 non-null  int64  
 1   SK_ID_CURR                          3840312 non-null  int64  
 2   MONTHS_BALANCE                      3840312 non-null  int64  
 3   AMT_BALANCE                         3840312 non-null  float64
 4   AMT_CREDIT_LIMIT_ACTUAL             3840312 non-null  int64  
 5   AMT_DRAWINGS_ATM_CURRENT            3090496 non-null  float64
 6   AMT_DRAWINGS_CURRENT                3840312 non-null  float64
 7   AMT_DRAWINGS_OTHER_CURRENT          3090496 non-null  float64
 8   AMT_DRAWINGS_POS_CURRENT            3090496 non-null  float64
 9   AMT_INST_MIN_REGULARITY             3535076 non-null  float64
 10  AMT_PAYMENT_CURRENT                 3072324 non-null  float64
 11  AMT_PAYMENT

In [150]:
for col in [col for col in cc_bal.columns if col not in ['SK_ID_PREV','SK_ID_CURR']]:
    print(col)
    agg_mean = cc_bal.groupby(['SK_ID_PREV'])[col].mean()
    prev_app_tmp['CC_'+col+'_mean'] = prev_app_tmp['SK_ID_PREV'].map(lambda x: agg_mean[x] if x in agg_mean.index else np.nan)
prev_app_tmp.info()

MONTHS_BALANCE
AMT_BALANCE
AMT_CREDIT_LIMIT_ACTUAL
AMT_DRAWINGS_ATM_CURRENT
AMT_DRAWINGS_CURRENT
AMT_DRAWINGS_OTHER_CURRENT
AMT_DRAWINGS_POS_CURRENT
AMT_INST_MIN_REGULARITY
AMT_PAYMENT_CURRENT
AMT_PAYMENT_TOTAL_CURRENT
AMT_RECEIVABLE_PRINCIPAL
AMT_RECIVABLE
AMT_TOTAL_RECEIVABLE
CNT_DRAWINGS_ATM_CURRENT
CNT_DRAWINGS_CURRENT
CNT_DRAWINGS_OTHER_CURRENT
CNT_DRAWINGS_POS_CURRENT
CNT_INSTALMENT_MATURE_CUM
SK_DPD
SK_DPD_DEF
NAME_CONTRACT_STATUS_Approved
NAME_CONTRACT_STATUS_Completed
NAME_CONTRACT_STATUS_Demand
NAME_CONTRACT_STATUS_Refused
NAME_CONTRACT_STATUS_Sent proposal
NAME_CONTRACT_STATUS_Signed
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1670214 entries, 0 to 1670213
Data columns (total 69 columns):
 #   Column                                               Non-Null Count    Dtype  
---  ------                                               --------------    -----  
 0   SK_ID_CURR                                           1670214 non-null  int64  
 1   SK_ID_PREV                  

In [189]:
prev_app_tmp['CC_AMT_BALANCE_mean'].describe()
prev_app_tmp['CC_AMT_BALANCE_mean'].info()


<class 'pandas.core.series.Series'>
RangeIndex: 1670214 entries, 0 to 1670213
Series name: CC_AMT_BALANCE_mean
Non-Null Count  Dtype  
--------------  -----  
92935 non-null  float64
dtypes: float64(1)
memory usage: 12.7 MB


Aggregating PREV_APPLICATION ==> APPLICATION_TRAIN

In [151]:
for col in [col for col in prev_app_tmp.columns if col not in ['SK_ID_CURR','SK_ID_PREV']]:
    print(col)
    agg_mean = prev_app_tmp.groupby(['SK_ID_CURR'])[col].mean()
    app_train_tmp['PREV_'+col+'_mean'] = prev_app_tmp['SK_ID_CURR'].map(lambda x: agg_mean[x] if x in agg_mean.index else np.nan)
app_train_tmp.info(verbose=True, show_counts=True)

AMT_ANNUITY
AMT_APPLICATION
AMT_CREDIT
DAYS_DECISION
CNT_PAYMENT
DAYS_FIRST_DUE
DAYS_LAST_DUE_1ST_VERSION
DAYS_LAST_DUE
DAYS_TERMINATION
NFLAG_INSURED_ON_APPROVAL
NAME_CONTRACT_TYPE_Consumer loans
NAME_CONTRACT_TYPE_Revolving loans
NAME_CONTRACT_TYPE_XNA
NAME_CONTRACT_STATUS_Canceled
NAME_CONTRACT_STATUS_Refused
NAME_CONTRACT_STATUS_Unused offer
NAME_CLIENT_TYPE_Refreshed
NAME_CLIENT_TYPE_Repeater
NAME_CLIENT_TYPE_XNA
NAME_YIELD_GROUP_high
NAME_YIELD_GROUP_low_action
NAME_YIELD_GROUP_low_normal
NAME_YIELD_GROUP_middle
POS_MONTHS_BALANCE_mean
POS_CNT_INSTALMENT_mean
POS_CNT_INSTALMENT_FUTURE_mean
POS_SK_DPD_mean
POS_SK_DPD_DEF_mean
POS_NAME_CONTRACT_STATUS_Amortized debt_mean
POS_NAME_CONTRACT_STATUS_Approved_mean
POS_NAME_CONTRACT_STATUS_Canceled_mean
POS_NAME_CONTRACT_STATUS_Completed_mean
POS_NAME_CONTRACT_STATUS_Demand_mean
POS_NAME_CONTRACT_STATUS_Returned to the store_mean
POS_NAME_CONTRACT_STATUS_Signed_mean
IP_NUM_INSTALMENT_VERSION_mean
IP_NUM_INSTALMENT_NUMBER_mean
IP_DAYS_INS

C:\Users\nanda\AppData\Local\Temp\ipykernel_8684\4263693235.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  app_train_tmp['PREV_'+col+'_mean'] = prev_app_tmp['SK_ID_CURR'].map(lambda x: agg_mean[x] if x in agg_mean.index else np.nan)


CC_NAME_CONTRACT_STATUS_Demand_mean


C:\Users\nanda\AppData\Local\Temp\ipykernel_8684\4263693235.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  app_train_tmp['PREV_'+col+'_mean'] = prev_app_tmp['SK_ID_CURR'].map(lambda x: agg_mean[x] if x in agg_mean.index else np.nan)


CC_NAME_CONTRACT_STATUS_Refused_mean


C:\Users\nanda\AppData\Local\Temp\ipykernel_8684\4263693235.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  app_train_tmp['PREV_'+col+'_mean'] = prev_app_tmp['SK_ID_CURR'].map(lambda x: agg_mean[x] if x in agg_mean.index else np.nan)


CC_NAME_CONTRACT_STATUS_Sent proposal_mean


C:\Users\nanda\AppData\Local\Temp\ipykernel_8684\4263693235.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  app_train_tmp['PREV_'+col+'_mean'] = prev_app_tmp['SK_ID_CURR'].map(lambda x: agg_mean[x] if x in agg_mean.index else np.nan)


CC_NAME_CONTRACT_STATUS_Signed_mean
<class 'pandas.core.frame.DataFrame'>
Int64Index: 307500 entries, 0 to 307510
Data columns (total 146 columns):
 #    Column                                                         Non-Null Count   Dtype  
---   ------                                                         --------------   -----  
 0    SK_ID_CURR                                                     307500 non-null  int64  
 1    NAME_CONTRACT_TYPE                                             307500 non-null  int64  
 2    CODE_GENDER                                                    307500 non-null  int64  
 3    FLAG_OWN_REALTY                                                307500 non-null  int64  
 4    CNT_CHILDREN                                                   307500 non-null  int64  
 5    AMT_INCOME_TOTAL                                               307500 non-null  float64
 6    AMT_CREDIT                                                     307500 non-null  float64
 7    

C:\Users\nanda\AppData\Local\Temp\ipykernel_8684\4263693235.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  app_train_tmp['PREV_'+col+'_mean'] = prev_app_tmp['SK_ID_CURR'].map(lambda x: agg_mean[x] if x in agg_mean.index else np.nan)


In [167]:
train_agg_dataset_dir = win_dir+'train_agg.csv'
if (os.path.exists(train_agg_dataset_dir)):
    print('File already exist!')
else:
    print('File doesn\'t exist, Export file at \'{}\''.format(train_agg_dataset_dir))
    app_train_tmp.to_csv(train_agg_dataset_dir, index=False)

File doesn't exist, Export file at 'D:/Projects/Dataset/HCI/home-credit-default-risk/train_agg.csv'


In [168]:
df = pd.read_csv(train_agg_dataset_dir)
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307500 entries, 0 to 307499
Data columns (total 146 columns):
 #    Column                                                         Non-Null Count   Dtype  
---   ------                                                         --------------   -----  
 0    SK_ID_CURR                                                     307500 non-null  int64  
 1    NAME_CONTRACT_TYPE                                             307500 non-null  int64  
 2    CODE_GENDER                                                    307500 non-null  int64  
 3    FLAG_OWN_REALTY                                                307500 non-null  int64  
 4    CNT_CHILDREN                                                   307500 non-null  int64  
 5    AMT_INCOME_TOTAL                                               307500 non-null  float64
 6    AMT_CREDIT                                                     307500 non-null  float64
 7    AMT_ANNUITY                         

In [180]:
df[['BUR_DAYS_ENDDATE_FACT_mean','BUR_AMT_ANNUITY_mean','PREV_CC_MONTHS_BALANCE_mean_mean','PREV_CC_AMT_BALANCE_mean_mean']].describe().T

,count,mean,std,min,25%,50%,75%,max
BUR_DAYS_ENDDATE_FACT_mean,295715.0,-986.500209,454.266668,-8376.500000,-1270.683333,-970.500000,-666.690476,0.000000e+00
BUR_AMT_ANNUITY_mean,142976.0,14574.097756,83725.427796,0.000000,2163.937500,6694.681500,13836.000000,8.516460e+06
PREV_CC_MONTHS_BALANCE_mean_mean,119166.0,-15.553583,13.491078,-50.000000,-20.000000,-10.000000,-5.500000,-1.000000e+00
PREV_CC_AMT_BALANCE_mean_mean,119166.0,76279.558900,113303.807289,-2930.232558,0.000000,28188.842321,110401.800144,9.086101e+05


### *Data Preparation*

In [211]:
clean_df = df.copy()
clean_df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307500 entries, 0 to 307499
Data columns (total 146 columns):
 #    Column                                                         Non-Null Count   Dtype  
---   ------                                                         --------------   -----  
 0    SK_ID_CURR                                                     307500 non-null  int64  
 1    NAME_CONTRACT_TYPE                                             307500 non-null  int64  
 2    CODE_GENDER                                                    307500 non-null  int64  
 3    FLAG_OWN_REALTY                                                307500 non-null  int64  
 4    CNT_CHILDREN                                                   307500 non-null  int64  
 5    AMT_INCOME_TOTAL                                               307500 non-null  float64
 6    AMT_CREDIT                                                     307500 non-null  float64
 7    AMT_ANNUITY                         

In [212]:
many_na = [col for col in clean_df.columns if col.startswith('PREV_CC_')] + ['BUR_AMT_ANNUITY_mean','BUR_AMT_CREDIT_MAX_OVERDUE_mean']
clean_df[many_na].describe().T

,count,mean,std,min,25%,50%,75%,max
PREV_CC_MONTHS_BALANCE_mean_mean,119166.0,-1.555358e+01,13.491078,-50.000000,-20.000000,-10.000000,-5.500000,-1.000000e+00
PREV_CC_AMT_BALANCE_mean_mean,119166.0,7.627956e+04,113303.807289,-2930.232558,0.000000,28188.842321,110401.800144,9.086101e+05
PREV_CC_AMT_CREDIT_LIMIT_ACTUAL_mean_mean,119166.0,2.253843e+05,192702.158297,0.000000,93750.000000,171000.000000,270942.028986,1.350000e+06
PREV_CC_AMT_DRAWINGS_ATM_CURRENT_mean_mean,80022.0,1.450993e+04,23468.676613,0.000000,1626.506024,6994.565217,19316.304878,9.045000e+05
PREV_CC_AMT_DRAWINGS_CURRENT_mean_mean,119166.0,1.558967e+04,28364.394318,0.000000,0.000000,5003.796715,21604.517386,1.616206e+06
PREV_CC_AMT_DRAWINGS_OTHER_CURRENT_mean_mean,80022.0,7.047382e+02,4833.264328,0.000000,0.000000,0.000000,0.000000,2.250000e+05
PREV_CC_AMT_DRAWINGS_POS_CURRENT_mean_mean,80022.0,9.791558e+03,24213.348889,0.000000,0.000000,1358.932500,11409.908906,1.616206e+06
PREV_CC_AMT_INST_MIN_REGULARITY_mean_mean,119166.0,3.757453e+03,5352.649301,0.000000,0.000000,1686.802500,5436.845357,4.301413e+04
PREV_CC_AMT_PAYMENT_CURRENT_mean_mean,79978.0,2.094921e+04,30917.826747,0.000000,6129.407181,12444.437144,25213.786875,1.593111e+06
PREV_CC_AMT_PAYMENT_TOTAL_CURRENT_mean_mean,119166.0,1.149124e+04,23955.575502,0.000000,0.000000,4593.512500,13909.370233,1.591837e+06


Dealing Null Value using Radical Way

In [213]:
# clean_df[many_na] = clean_df[many_na].fillna(0)
clean_df.dropna(inplace=True)
clean_df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24664 entries, 793 to 306789
Data columns (total 146 columns):
 #    Column                                                         Non-Null Count  Dtype  
---   ------                                                         --------------  -----  
 0    SK_ID_CURR                                                     24664 non-null  int64  
 1    NAME_CONTRACT_TYPE                                             24664 non-null  int64  
 2    CODE_GENDER                                                    24664 non-null  int64  
 3    FLAG_OWN_REALTY                                                24664 non-null  int64  
 4    CNT_CHILDREN                                                   24664 non-null  int64  
 5    AMT_INCOME_TOTAL                                               24664 non-null  float64
 6    AMT_CREDIT                                                     24664 non-null  float64
 7    AMT_ANNUITY                                 

### **Split Data for Training and Testing**

In [234]:
var_columns = [col for col in clean_df.columns if col != 'TARGET']
x, y = clean_df[var_columns].copy(), clean_df.TARGET.copy()
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.3, random_state=42)
ytrain.value_counts(), ytrain.value_counts(normalize=True)*100

(0    15985
 1     1279
 Name: TARGET, dtype: int64,
 0    92.59152
 1     7.40848
 Name: TARGET, dtype: float64)

In [235]:
scaler = get_minmax_scaler(xtrain, xtrain.columns)
xtrain = apply_transform(scaler, xtrain)
xtest = apply_transform(scaler, xtest)

Dealing ImBalance Data, Give a Balance data option for Balance Data Scenario

In [236]:
xtrain_balance, ytrain_balance = xtrain.copy(), ytrain.copy()
# X_train_balance, y_train_balance  = over_sampling.SMOTE(sampling_strategy=1.0).fit_resample(X_train_balance, y_train_balance)
xtrain_balance, ytrain_balance  = over_sampling.SMOTE().fit_resample(xtrain_balance, ytrain_balance)
ytrain_balance.value_counts()

0    15985
1    15985
Name: TARGET, dtype: int64

## **Modeling**

In [237]:
def eval_classification(model, xtrain, ytrain, xtest, ytest, threshold=0.5):
    ypred_proba_test = model.predict_proba(xtest)
    ypred_proba_train = model.predict_proba(xtrain)
    ypred_datatest = np.where(ypred_proba_test[:,1]>=threshold, 1, 0 )
    ypred_datatrain = np.where(ypred_proba_train[:,1]>=threshold, 1, 0 )
    cm = confusion_matrix(ytest, ypred_datatest)
    tn, fp, fn, tp = cm.ravel()
    print("Accuracy (Test Set): %.2f" % accuracy_score(ytest, ypred_datatest))
    print("Precision (Test Set): %.2f" % precision_score(ytest, ypred_datatest))
    # print("Precision (Train Set): %.2f" % precision_score(ytrain, ypred_datatrain))
    print("Recall (Test Set): %.2f" % recall_score(ytest, ypred_datatest))
    # print("Recall (Train Set): %.2f" % recall_score(ytrain, ypred_datatrain))
    print("F1-Score (Test Set): %.2f" % f1_score(ytest, ypred_datatest))
    # print("F1-Score (Train Set): %.2f" % f1_score(ytrain, ypred))
    print("AUC (test-proba): %.2f" % roc_auc_score(ytest, ypred_proba_test[:,1] ))
    print("AUC (train-proba): %.2f" % roc_auc_score(ytrain, ypred_proba_train[:,1] ))
    print('tp : {}\nfp  : {}\nfn  : {}\ntn  : {}\n'.format(tp, fp, fn, tn))

### **RandomForest**

In [238]:
rf = RandomForestClassifier(random_state=42)
temp_rf = rf.fit(xtrain, ytrain.values.ravel())
eval_classification(rf, xtrain, ytrain, xtest, ytest)

Accuracy (Test Set): 0.92
Precision (Test Set): 0.00
Recall (Test Set): 0.00
F1-Score (Test Set): 0.00
AUC (test-proba): 0.56
AUC (train-proba): 1.00
tp : 0
fp  : 0
fn  : 601
tn  : 6799



C:\SYS\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [239]:
temp_rf = rf.fit(xtrain_balance, ytrain_balance.values.ravel())
eval_classification(rf, xtrain_balance, ytrain_balance, xtest, ytest)

Accuracy (Test Set): 0.91
Precision (Test Set): 0.11
Recall (Test Set): 0.02
F1-Score (Test Set): 0.03
AUC (test-proba): 0.55
AUC (train-proba): 1.00
tp : 10
fp  : 80
fn  : 591
tn  : 6719



### **XGBoost**

In [240]:
xgb = XGBClassifier(random_state=42)
temp_xgb = xgb.fit(xtrain, ytrain.values.ravel())
eval_classification(xgb, xtrain, ytrain, xtest, ytest)

Accuracy (Test Set): 0.92
Precision (Test Set): 0.18
Recall (Test Set): 0.00
F1-Score (Test Set): 0.01
AUC (test-proba): 0.56
AUC (train-proba): 1.00
tp : 2
fp  : 9
fn  : 599
tn  : 6790



### **Logistic Regression**

In [243]:
lr = LogisticRegression()
temp_lr = lr.fit(xtrain, ytrain.values.ravel())
eval_classification(temp_lr, xtrain, ytrain, xtest, ytest)

Accuracy (Test Set): 0.92
Precision (Test Set): 0.00
Recall (Test Set): 0.00
F1-Score (Test Set): 0.00
AUC (test-proba): 0.61
AUC (train-proba): 0.66
tp : 0
fp  : 0
fn  : 601
tn  : 6799



C:\SYS\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\SYS\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [244]:
temp_lr = lr.fit(xtrain_balance, ytrain_balance.values.ravel())
eval_classification(temp_lr, xtrain_balance, ytrain_balance, xtest, ytest)

Accuracy (Test Set): 0.59
Precision (Test Set): 0.11
Recall (Test Set): 0.54
F1-Score (Test Set): 0.18
AUC (test-proba): 0.60
AUC (train-proba): 0.68
tp : 325
fp  : 2761
fn  : 276
tn  : 4038



C:\SYS\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
